In [1]:
import glob
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import csv

pd.options.display.float_format = '{:.5f}'.format
pd.set_option('display.max_columns', None)

In [2]:
use_counties = ['台北市', '高雄市', '新北市', '桃園市', '台中市', '台南市', '苗栗縣', '新竹縣', '基隆市',
       '屏東縣', '新竹市', '宜蘭縣', '花蓮縣', '嘉義市', '金門縣', '嘉義縣', '彰化縣', '雲林縣']
dfs = []
for pathto_file in glob.glob('../外部資料集/實價登錄/*/*_c.csv'):
    df = pd.read_csv(pathto_file, on_bad_lines='skip', skiprows=[1], quoting=csv.QUOTE_NONE)
    df['單價元平方公尺'].fillna(0)
    df['土地位置建物門牌'] = df['土地位置建物門牌'].apply(lambda x: (x.replace('臺北市', '台北市').replace('臺中市', '台中市').replace('臺南市', '台南市')))
    county = df['土地位置建物門牌'].apply(lambda x: (x+'123')[:3]).value_counts().index[0]
    if county not in use_counties:
        continue
    df['縣市'] = county
    df['source'] = pathto_file.split('/')[-2]
    avg_price = df.query('單價元平方公尺 > 1')['單價元平方公尺'].median()
    avg_price = avg_price*0.1
    df = df.query(f'單價元平方公尺 > {avg_price}')
    dfs.append(df)
df = pd.concat(dfs).reset_index(drop=True)
df.sample()

,鄉鎮市區,交易標的,土地位置建物門牌,土地面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,租賃年月日,租賃筆棟數,租賃層次,總樓層數,建物型態,主要用途,主要建材,建築完成年月,建物總面積平方公尺,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,有無附傢俱,總額元,單價元平方公尺,車位類別,車位面積平方公尺,車位總額元,備註,編號,縣市,source
90171,新莊區,建物,新北市新莊區中平路１８４巷２８號五樓,0.00000,住,NaN,NaN,1110222,土地0建物1車位0,五層,10.00000,套房(1房1廳1衛),集合住宅,鋼筋混凝土造,1090518.00000,49.67000,0,0,0,無,有,有,16000,322.00000,NaN,0.00000,0,NaN,RPQQMLLKLHHGFBF67CC,新北市,2022Q2


In [3]:
use_sources = ['2021Q1', '2021Q2', '2021Q3', '2021Q4', '2022Q1', '2022Q2', '2022Q3', '2022Q4', '2023Q1', '2023Q2', '2023Q3']

print(df.shape)
df = df[df['source'].isin(use_sources)].copy().reset_index(drop=True)
print(df.shape)

(177848, 31)
(146225, 31)


In [4]:
df.query('主要用途 == "住家用"')['單價元平方公尺'].describe()

count   83268.00000
mean      230.87904
std       152.86776
min        16.00000
25%       137.00000
50%       187.00000
75%       271.00000
max      7084.00000
Name: 單價元平方公尺, dtype: float64

In [5]:
set(use_counties) - set(df['縣市'].unique().tolist())

set()

In [6]:
df['縣市'].value_counts()

縣市
桃園市    28931
新北市    26734
台北市    24492
高雄市    21775
台中市    18112
台南市    11355
基隆市     3081
屏東縣     2102
新竹縣     1705
新竹市     1584
嘉義市     1543
宜蘭縣     1371
花蓮縣     1280
雲林縣      646
苗栗縣      614
彰化縣      501
嘉義縣      385
金門縣       14
Name: count, dtype: int64

In [7]:
df.groupby('鄉鎮市區')['單價元平方公尺'].mean()

鄉鎮市區
七堵區   156.10317
七股區   136.53333
三峽區   223.67344
三星鄉    98.55556
三民區   207.13559
         ...   
麻豆區   151.48387
鼓山區   222.58997
龍井區   199.65657
龍潭區   137.06264
龜山區   201.05920
Name: 單價元平方公尺, Length: 245, dtype: float64

In [8]:
# df = df.query('主建物面積 >= 2').copy().reset_index(drop=True)
# df = df.query('主建物面積 <= 2000').copy().reset_index(drop=True)
# df = df.query('陽台面積 <= 100').copy().reset_index(drop=True)
# df = df.query('單價元平方公尺 >= 1000')

col = '主建物面積'
df[col] = 0


col = '陽台面積'
df[col] = 0

col = '附屬建物面積'
df[col] = 0

col = '車位面積'
df[col] = df['車位面積平方公尺'].values
scaler = StandardScaler()
scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
print(df[col].describe())

col = '建物面積'
df[col] = df['建物總面積平方公尺'].values
scaler = StandardScaler()
scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
print(df[col].describe())

col = '土地面積'
df[col] = 0

# col = '單價'
# df[col] = df['單價元平方公尺'].values
# scaler = StandardScaler()
# scaler.fit(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1))
# df[col] = scaler.transform(df[col].apply(np.sqrt).to_numpy().reshape(-1, 1)) + 2
# print(df[col].describe())

count   146225.00000
mean         0.00000
std          1.00000
min         -0.34770
25%         -0.34770
50%         -0.34770
75%         -0.34770
max         27.79752
Name: 車位面積, dtype: float64
count   146225.00000
mean        -0.00000
std          1.00000
min         -2.27102
25%         -0.57138
50%         -0.03895
75%          0.41240
max         28.83206
Name: 建物面積, dtype: float64


In [9]:
def get_year_from_str(x):
    try:
        x = int(x)
        if x < 1000101:
            return int(str(x)[0:2])
        else:
            return int(str(x)[0:3])
    except:
        return -1

def get_month_from_str(x):
    try:
        x = int(x)
        if x < 1000101:
            return int(str(x)[2:4])
        else:
            return int(str(x)[3:5])
    except:
        return -1

In [10]:
df['Trade_Year'] = df['租賃年月日'].apply(get_year_from_str).astype(int) + 1911
df['Trade_Month'] = df['租賃年月日'].apply(get_month_from_str).astype(int)
df['Trade_YearMonth'] = df['Trade_Year']*100 + df['Trade_Month']

In [11]:
df = df[~df['建築完成年月'].isna()].copy().reset_index(drop=True)

In [12]:
df['Build_Year'] = df['建築完成年月'].apply(get_year_from_str).astype(int) + 1911
df['Build_Month'] = df['建築完成年月'].apply(get_month_from_str).astype(int)

In [13]:
df = df.query('Build_Year > 1910')
df = df.query('Build_Month > 0 and Build_Month <= 12').copy().reset_index(drop=True)

In [14]:
pd.set_option('display.max_columns', None)
df = df.query('Trade_Year >= 2021 and Trade_Year <= 2022')

In [15]:
df['屋齡'] = df['Trade_Year'] - df['Build_Year'] + (df['Trade_Month'] - df['Build_Month'])/12
df = df.query('屋齡 >= 0 and 屋齡 <= 60.1').copy().reset_index(drop=True)
df['屋齡'].describe()

count   94435.00000
mean       23.46014
std        14.39471
min         0.00000
25%        11.16667
50%        25.08333
75%        34.33333
max        60.08333
Name: 屋齡, dtype: float64

In [16]:
df['交易標的'].value_counts()

交易標的
建物              69777
房地(土地+建物)       17328
房地(土地+建物)+車位     7330
Name: count, dtype: int64

In [17]:
df['建物型態'].value_counts()

建物型態
住宅大樓(11層含以上有電梯)    35737
華廈(10層含以下有電梯)      15700
公寓(5樓含以下無電梯)       13262
套房(1房1廳1衛)         10314
透天厝                 9633
店面(店鋪)              4735
辦公商業大樓              3554
廠辦                  1130
其他                   370
Name: count, dtype: int64

In [18]:
df = df[~df['總樓層數'].isna()].copy()
df = df[~df['總樓層數'].isna()].copy()
df = df.query('交易標的 != "土地"').copy()
df = df.reset_index(drop=True)
df['交易標的'].value_counts()

交易標的
建物              69777
房地(土地+建物)       17328
房地(土地+建物)+車位     7330
Name: count, dtype: int64

In [19]:
df.sample(10)

,鄉鎮市區,交易標的,土地位置建物門牌,土地面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,租賃年月日,租賃筆棟數,租賃層次,總樓層數,建物型態,主要用途,主要建材,建築完成年月,建物總面積平方公尺,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,有無附傢俱,總額元,單價元平方公尺,車位類別,車位面積平方公尺,車位總額元,備註,編號,縣市,source,主建物面積,陽台面積,附屬建物面積,車位面積,建物面積,土地面積,Trade_Year,Trade_Month,Trade_YearMonth,Build_Year,Build_Month,屋齡
31341,萬華區,建物,台北市萬華區貴陽街二段３３號六樓之８,0.00000,商,NaN,NaN,1101201,土地0建物1車位0,六層,19.00000,住宅大樓(11層含以上有電梯),一般事務所（Ｇ２）,鋼筋混凝土造,1061127.00000,83.84000,2,2,1,有,有,有,36000,429.00000,NaN,0.00000,0,NaN,RPVNMLQKKIGGFBA77CC,台北市,2022Q1,0,0,0,-0.34770,-0.14829,0,2021,12,202112,2017,11,4.08333
17358,桃園區,建物,桃園市桃園區樹仁三街７６號五樓,0.00000,住,NaN,NaN,1100520,土地0建物1車位0,五層,14.00000,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,941228.00000,150.90000,3,2,2,有,無,有,22500,149.00000,NaN,0.00000,0,社會住宅另有提供車位，無償使用,RPPSMLOLNHGGFAH38CC,桃園市,2021Q3,0,0,0,-0.34770,0.57681,0,2021,5,202105,2005,12,15.41667
16373,淡水區,房地(土地+建物)+車位,新北市淡水區新民街一段３之１號七樓,17.55000,住,NaN,NaN,1100630,土地1建物1車位1,七層,17.00000,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,981225.00000,86.25000,3,2,1,有,有,有,20500,238.00000,坡道平面,27.65000,0,NaN,RPSNMLKKPHGGFEF08CC,新北市,2021Q3,0,0,0,2.52518,-0.11800,0,2021,6,202106,2009,12,11.50000
67129,苗栗市,建物,苗栗縣苗栗市新庄街３９巷２２號,115.00000,NaN,鄉村區,乙種建築用地,1100823,土地0建物1車位0,全,3.00000,透天厝,住家用,鋼筋混凝土造,620115.00000,73.84000,2,1,2,有,無,有,5800,79.00000,NaN,0.00000,0,水費由承租人負擔每月200元電費、瓦斯費由承租人負擔,RPPOMLOJPHHGFBK48CC,苗栗縣,2022Q3,0,0,0,-0.34770,-0.27890,0,2021,8,202108,1973,1,48.58333
25198,左營區,房地(土地+建物),高雄市左營區榮總路２６７號,165.62000,商,NaN,NaN,1100930,土地1建物1車位0,全,5.00000,透天厝,住商用,鋼筋混凝土造,970701.00000,96.41000,2,0,1,有,無,無,50000,519.00000,NaN,0.00000,0,NaN,RPRPMLRLIIGGFEE28CC,高雄市,2021Q4,0,0,0,-0.34770,0.00528,0,2021,9,202109,2008,7,13.16667
21591,北區,建物,台中市北區福龍街９７號１１樓之３,0.00000,商,NaN,NaN,1100801,土地0建物1車位0,十一層,12.00000,住宅大樓(11層含以上有電梯),商業用,鋼筋混凝土造,840502.00000,34.45000,1,0,1,有,無,有,10800,313.00000,NaN,0.00000,0,住宅及都市更新中心110年社會住宅包租代管第三期計畫地號：西區 後?子段 0097-0021,RPSQMLRKQHGGFBB48CC,台中市,2021Q4,0,0,0,-0.34770,-0.91032,0,2021,8,202108,1995,5,26.25000
34304,北區,建物,台南市北區公園南路３４１號三樓之２,0.00000,商,NaN,NaN,1101216,土地0建物1車位0,三層,14.00000,住宅大樓(11層含以上有電梯),集合住宅,鋼筋混凝土造,971219.00000,98.04000,2,1,1,有,有,有,17800,182.00000,NaN,0.00000,0,台南市社會住宅包租代管第二期計畫，租金不含停車位及管理費,RPSOMLLLKIGGFAD37CC,台南市,2022Q1,0,0,0,-0.34770,0.02444,0,2021,12,202112,2008,12,13.00000
24867,新興區,建物,高雄市新興區七賢一路２１６號９樓之２,0.00000,商,NaN,NaN,1100918,土地0建物1車位1,九層,13.00000,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,830302.00000,141.33000,3,2,2,有,有,有,16000,113.00000,坡道機械,0.00000,0,車位面積無法分算。,RPQNMLQJIIGGFBE37CC,高雄市,2021Q4,0,0,0,-0.34770,0.48502,0,2021,9,202109,1994,3,27.50000
65672,楊梅區,建物,桃園市楊梅區中山南路６００之１２號１１樓之１,0.00000,住,NaN,NaN,1110609,土地0建物1車位0,十一層,12.00000,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,840529.00000,111.84000,3,2,2,有,有,有,15900,142.00000,NaN,0.00000,0,社會住宅 管理費另計＄600,RPQNMLLKPHHGFDH18CC,桃園市,2022Q3,0,0,0,-0.34770,0.18068,0,2022,6,202206,1995,5,27.08333
35230,左營區,建物,高雄市左營區文學路２６７號,0.00000,住,NaN,NaN,1101122,土地0建物1車位0,一層,14.00000,其他,店舖,鋼筋混凝土,1071031.00000,171.26000,0,1,1,有,有,無,55000,321.00000,NaN,0.00000,0,承租用途做倉庫使用。,RPORMLSJKIGGFEE28CC,高雄市,2022Q1,0,0,0,-0.34770,0.76285,0,2021,11,202111,2018,10,3.08333


In [20]:
floor_dict = {
    '二十': 20,
    '三十': 30,
    '四十': 40,
    '五十': 50,
    '六十': 60,
    '七十': 70,
    '八十': 80,
    '九十': 90,
}
floor_dict2 = {
    '一': 1,
    '二': 2,
    '三': 3,
    '四': 4,
    '五': 5,
    '六': 6,
    '七': 7,
    '八': 8,
    '九': 9,
}
def get_floor(x):
    x = str(x)
    floor = 0
    for k, v in floor_dict.items():
        if x.startswith(k):
            x = x.replace(k, '')
            floor += v
    if x.startswith('十'):
        x = x.replace('十', '')
        floor += 10
    for k, v in floor_dict2.items():
        if x.startswith(k):
            x = x.replace(k, '')
            floor += v
    return floor

df['total_floor'] = df['總樓層數'].astype(int)
df['floor'] = df['租賃層次'].apply(get_floor)

In [21]:
def get_road_name(row):
    road_name = row['土地位置建物門牌']
    if road_name.startswith(row['縣市']):
        road_name = road_name.replace(row['縣市'], '')
    if road_name.startswith(row['鄉鎮市區']):
        road_name = road_name.replace(row['鄉鎮市區'], '')
    if '段' in road_name:
        road_name = road_name.split('段')[0] + '段'
    elif '路' in road_name:
        road_name = road_name.split('路')[0] + '路'
    elif '街' in road_name:
        road_name = road_name.split('街')[0] + '街'
    else:
        road_name = ""
    return road_name
    
df['路名'] = df.apply(lambda row: get_road_name(row), axis=1)

In [22]:
df = df.query('floor > 0').reset_index(drop=True)
df = df.query('total_floor > 0').reset_index(drop=True)
df = df.query('total_floor >= floor').reset_index(drop=True)
df = df.query('路名 != ""').reset_index(drop=True)

In [23]:
df.sample(5)

,鄉鎮市區,交易標的,土地位置建物門牌,土地面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,租賃年月日,租賃筆棟數,租賃層次,總樓層數,建物型態,主要用途,主要建材,建築完成年月,建物總面積平方公尺,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,有無附傢俱,總額元,單價元平方公尺,車位類別,車位面積平方公尺,車位總額元,備註,編號,縣市,source,主建物面積,陽台面積,附屬建物面積,車位面積,建物面積,土地面積,Trade_Year,Trade_Month,Trade_YearMonth,Build_Year,Build_Month,屋齡,total_floor,floor,路名
61679,佳里區,房地(土地+建物),台南市佳里區佳興六街６１號四樓之５,37.60000,NaN,鄉村區,乙種建築用地,1110923,土地1建物1車位0,四層,5.00000,華廈(10層含以下有電梯),集合住宅,鋼筋混凝土造,1090416.00000,108.11000,2,1,1,有,有,有,16500,153.00000,NaN,0.00000,0,NaN,RPRNMLKMRHHGFGD48CC,台南市,2022Q4,0,0,0,-0.34770,0.13945,0,2022,9,202209,2020,4,2.41667,5,4,佳興六街
69115,桃園區,建物,桃園市桃園區國聖一街１２號七樓之３,0.00000,住,NaN,NaN,1111022,土地0建物1車位1,七層,16.00000,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,951208.00000,162.71000,3,2,2,有,有,有,22000,135.00000,坡道平面,35.60000,0,NaN,RPTPMLSLIIHGFAH58CC,桃園市,2022Q4,0,0,0,2.91213,0.68615,0,2022,10,202210,2006,12,15.83333,16,7,國聖一街
56186,八德區,建物,桃園市八德區建德路２６０號二樓,0.00000,商,NaN,NaN,1110412,土地0建物1車位1,二層,14.00000,住宅大樓(11層含以上有電梯),集合住宅,鋼筋混凝土造,1101022.00000,140.55000,3,2,2,有,有,有,18400,131.00000,坡道平面,0.00000,0,社會住宅 租賃車位一個,RPQOMLKKQHHGFFH48CC,桃園市,2022Q3,0,0,0,-0.34770,0.47741,0,2022,4,202204,2021,10,0.50000,14,2,建德路
61426,東區,建物,台南市東區東寧路２０之２號,0.00000,住,NaN,NaN,1110828,土地0建物1車位0,三層,4.00000,公寓(5樓含以下無電梯),住家用,鋼筋混凝土造,700930.00000,89.55000,3,2,1,有,無,有,17000,190.00000,NaN,0.00000,0,NaN,RPOOMLPJRHHGFCD08CC,台南市,2022Q4,0,0,0,-0.34770,-0.07720,0,2022,8,202208,1981,9,40.91667,4,3,東寧路
46013,中山區,建物,台北市中山區松江路３７２巷１２號１樓,0.00000,商,NaN,NaN,1110707,土地0建物1車位0,一層,7.00000,華廈(10層含以下有電梯),商業用,鋼筋混凝土造,731109.00000,125.09000,4,2,1,有,無,有,56889,455.00000,NaN,0.00000,0,NaN,RPQOMLRJPHHGFCA08DC,台北市,2022Q3,0,0,0,-0.34770,0.32185,0,2022,7,202207,1984,11,37.66667,7,1,松江路


In [24]:
df['總樓層數'] = df['total_floor']
df['移轉層次'] = df['floor']

In [25]:
def get_car_cnt(x):
    x = x.split('車位')[-1]
    if x.isdigit():
        return int(x)
    return -999999

In [26]:
df['車位個數'] = df['租賃筆棟數'].apply(get_car_cnt).astype(int)
df['車位個數'].describe()

count   79909.00000
mean        0.22522
std         0.58672
min         0.00000
25%         0.00000
50%         0.00000
75%         0.00000
max        72.00000
Name: 車位個數, dtype: float64

In [27]:
df['單價'] = df['單價元平方公尺'].apply(lambda x: x/124)
print(df['單價'].describe())

count   79909.00000
mean        1.99385
std         1.32217
min         0.12903
25%         1.18548
50%         1.60484
75%         2.35484
max        38.11290
Name: 單價, dtype: float64


In [28]:
df = df.query('單價 > 0').copy().reset_index(drop=True)

In [29]:
dfs_build = []
for pathto_file in glob.glob('../外部資料集/實價登錄/*/*_c_build.csv'):
    _df = pd.read_csv(pathto_file, skiprows=[1])
    dfs_build.append(_df)
df_build = pd.concat(dfs_build).reset_index(drop=True)

useages = ['住家用', '集合住宅', '其他', '店鋪', '商業用', '國民住宅', '住工用', '一般事務所', '住商用',
       '廠房', '工業用', '辦公室']
print(df_build.shape)
print(set(useages) - set(df_build.主要用途.unique().tolist()))
df_build = df_build[df_build['主要用途'].isin(useages)]
print(df_build.shape)
df_build = df_build.drop_duplicates(subset=['編號'], keep='first')
print(df_build.shape)

df_build_usage_map = dict(zip(df_build['編號'].values, df_build['主要用途'].values))

df['主要用途_from_info'] = df['編號'].map(df_build_usage_map)
df = df[~df['主要用途_from_info'].isna()].copy().reset_index(drop=True)
df = df[~df['主要用途'].isna()].copy().reset_index(drop=True)
print(df['主要用途'].value_counts())
print(df['主要用途_from_info'].value_counts())
df['主要用途'] = df['主要用途_from_info']

(365813, 9)
{'其他'}
(160769, 9)
(152574, 9)
主要用途
住家用                       49748
集合住宅                      12652
商業用                        4488
辦公室                        1422
見其他登記事項                    1301
住商用                        1184
工業用                         879
一般事務所                       688
國民住宅                        176
見其它登記事項                     164
廠房                           78
店鋪                           70
住工用                          22
見使用執照                         9
工商用                           5
防空避難室、停車空間                    1
停車空間                          1
工廠                            1
自用儲藏室﹐店舖                      1
自由職業事務所                       1
集合住宅，多戶住宅                     1
廠房、辦公室                        1
店房、人行道、集合住宅                   1
住宅，車庫                         1
人行道　住宅　店舖                     1
人行道．店舖                        1
防空避難室、會議室、康樂室、辦公室、員工宿舍        1
一般服務業                         1
Name: count, dtype: int64
主要用途_from_info
住家用      49736


In [30]:
print(df.shape)
df = df.query('移轉層次 >= 2 and 移轉層次 <= 46')
print(df.shape)
df = df.query('總樓層數 >= 2 and 總樓層數 <= 68')
print(df.shape)
df = df.query('車位個數 >= 0 and 車位個數 <= 3')
print(df.shape)

(72899, 50)
(67877, 50)
(67861, 50)
(67776, 50)


In [31]:
df['格局_房'] = df['建物現況格局-房']
df['格局_廳'] = df['建物現況格局-廳']
df['格局_衛'] = df['建物現況格局-衛']
df['格局_隔間'] = df['建物現況格局-隔間'].map({'有': True, '無': False})
#df['格局_電梯'] = df['電梯'].map({'有': True, '無': False})
df['格局_管理'] = df['有無管理組織'].map({'有': True, '無': False})
use_cols = ['縣市', '鄉鎮市區', '路名', '建物型態', '主要建材', '總樓層數', '移轉層次', 
            '格局_房', '格局_廳', '格局_衛', '格局_隔間', '格局_管理',
            '主要用途', '車位個數', '屋齡', '土地面積', '建物面積', '車位面積', '主建物面積', '陽台面積', '附屬建物面積', '單價']

df[use_cols].to_csv('../外部資料集/實價登錄/external_gov_data_lease.csv', index=False)
df[use_cols].head(10)

,縣市,鄉鎮市區,路名,建物型態,主要建材,總樓層數,移轉層次,格局_房,格局_廳,格局_衛,格局_隔間,格局_管理,主要用途,車位個數,屋齡,土地面積,建物面積,車位面積,主建物面積,陽台面積,附屬建物面積,單價
0,台北市,大安區,忠孝東路三段,住宅大樓(11層含以上有電梯),鋼筋混凝土造,13,10,3,2,2,True,True,住家用,0,26.33333,0,0.54940,-0.34770,0,0,0,2.66935
1,台北市,中山區,民生東路３段,辦公商業大樓,鋼筋混凝土造,12,2,0,0,0,False,True,商業用,0,35.00000,0,0.90285,-0.34770,0,0,0,2.58065
2,台北市,士林區,芝玉路二段,華廈(10層含以下有電梯),鋼筋混凝土造,10,3,3,2,2,True,False,住家用,1,22.91667,0,1.53134,-0.34770,0,0,0,3.00000
3,台北市,中山區,南京東路二段,套房(1房1廳1衛),鋼筋混凝土造,8,8,1,0,1,True,False,住商用,0,51.58333,0,-1.11698,-0.34770,0,0,0,3.02419
4,台北市,大安區,光復南路,住宅大樓(11層含以上有電梯),鋼筋混凝土造,12,3,2,2,1,True,True,商業用,0,31.25000,0,0.53843,-0.34770,0,0,0,3.12903
5,台北市,中山區,雙城街,套房(1房1廳1衛),鋼筋混凝土造,15,12,1,1,1,True,False,住家用,0,25.08333,0,-0.44574,-0.34770,0,0,0,2.33871
6,台北市,內湖區,瑞光路,辦公商業大樓,鋼筋混凝土造,10,3,2,0,0,True,True,工業用,1,22.33333,0,1.46949,-0.34770,0,0,0,2.76613
7,台北市,士林區,福德路,華廈(10層含以下有電梯),鋼筋混凝土造,8,6,1,1,1,True,True,住家用,0,9.00000,0,-0.92681,-0.34770,0,0,0,5.27419
8,台北市,萬華區,康定路,住宅大樓(11層含以上有電梯),見其他登記事項,26,19,1,1,1,True,True,住家用,0,4.83333,0,-0.53323,-0.34770,0,0,0,4.87903
9,台北市,內湖區,康樂街,公寓(5樓含以下無電梯),鋼筋混凝土造,5,3,3,2,1,True,False,住家用,0,36.50000,0,-0.17967,-0.34770,0,0,0,2.08065


In [32]:
df['格局_隔間'].sum()

63227

In [33]:
df['格局_隔間'].count()

67776

In [34]:
df['格局_隔間'].value_counts()

格局_隔間
True     63227
False     4549
Name: count, dtype: int64